# Pytorch-3D-Unet Image Labeler & Model Scoring

### Packages

In [1]:
!pip install napari-skimage-regionprops
!pip install matplotlib
!pip install tifffile
!pip install openpyxl

In [2]:
!pip install napari

### Imports

In [1]:
import napari
import pandas as pd
from napari_skimage_regionprops import regionprops_table, add_table, get_table
import numpy as np
from scipy import ndimage
from skimage import measure, morphology
import matplotlib.pyplot as plt
from datetime import datetime  # Import the datetime library


from cellpose import models, io
import cellpose

ModuleNotFoundError: No module named 'napari'

### Functions

In [2]:
'''
Function to get IDs of cells present in a certain % planes 
can be used for example to pinpoint the best / fully segmented ones 
or the truncated ones that need correcting! 
'''

def select_cells_by_integrity(z_planes, cells, percentage):
    target_n_planes = round ((percentage / 100 ) * z_planes)
    selected_cells = []
    IDs = np.unique(cells[:,1])
    counter_ID = np.zeros((np.max(IDs)+1, 1))
    for i in cells:
        counter_ID[i[1]] = counter_ID[i[1]] + 1
    count_good = 0
    count_bad = 0
    for cell_id, i in enumerate(counter_ID):
        if i >= target_n_planes:
            count_good = count_good + 1
        else:
            count_bad = count_bad + 1

    return count_good, count_bad
# function to filter labels based on a list of label ids one wants to keep (one frame)
def select_labels_oneframe(input_labels, ids_to_keep):
    mask = np.isin(input_labels, ids_to_keep)
    selected_labels = np.where(mask, input_labels, 0)
    return selected_labels

# function to filter labels based on a list of label ids one wants to keep (for a stack/movie)
def select_labels_stack(input_labels, ids_to_keep):
    picked_labels = []
    for i in range(0,len(input_labels.data)):
        current_frame = (input_labels.data[i])
        picked_labels.append(select_labels_oneframe(current_frame, ids_to_keep)) 
    return picked_labels



### ".h5" -> ".tif" converter.

In [7]:
# Install all the required packages
!pip install h5py
!pip install --upgrade pip
!pip install --upgrade Pillow
!pip install -U scikit-image
!pip install --upgrade scikit-image
!pip install tifffile

  Obtaining dependency information for h5py from https://files.pythonhosted.org/packages/15/9a/b5456e1acc4abb382938d4a730600823bfe77a4bbfd29140ccbf01ba5596/h5py-3.11.0-cp38-cp38-win_amd64.whl.metadata
   ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.0 MB 320.0 kB/s eta 0:00:10
   ---------------------------------------- 0.0/3.0 MB 217.9 kB/s eta 0:00:14
   ---------------------------------------- 0.0/3.0 MB 217.9 kB/s eta 0:00:14
    --------------------------------------- 0.0/3.0 MB 163.4 kB/s eta 0:00:19
    --------------------------------------- 0.0/3.0 MB 163.4 kB/s eta 0:00:19
    --------------------------------------- 0.1/3.0 MB 192.5 kB/s eta 0:00:16
   - -------------------------------------- 0.1/3.0 MB 238.1 kB/s eta 0:00:13
   - -------------------------------------- 0.1/3.0 MB 238.1 kB/s eta 0:00:13
   - --------------------------------

ERROR: To modify pip, please run the following command:
C:\Users\titom\anaconda3\envs\cellpose\python.exe -m pip install --upgrade pip


  Obtaining dependency information for Pillow from https://files.pythonhosted.org/packages/9c/52/d00d737c7e8143582090abf2d5eef9712422e7677ef721450abc1a1676bc/pillow-10.3.0-cp38-cp38-win_amd64.whl.metadata
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB 435.7 kB/s eta 0:00:06
   ---------------------------------------- 0.0/2.5 MB 435.7 kB/s eta 0:00:06
   ---------------------------------------- 0.0/2.5 MB 435.7 kB/s eta 0:00:06
    --------------------------------------- 0.0/2.5 MB 178.6 kB/s eta 0:00:14
    --------------------------------------- 0.1/2.5 MB 218.8 kB/s eta 0:00:12
   - -------------------------------------- 0.1/2.5 MB 270.5 kB/s eta 0:00:10
   - -------------------------------------- 0.1/2.5 MB 275.8 kB/s eta 0:00:09
   - -------------------------------------- 0.1/2.5 MB 297.7 kB/s eta 0:00:09
   - ----------------------------

In [3]:
# Import the libraries
import h5py
import os
import skimage
from skimage import io, transform, measure
import numpy as np
import tifffile

C:\Users\titom\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [4]:
# See which models you have in the folder named "models".
os.listdir(".")

#os.listdir(r"/media/maryam/Data/Alvaro/transferLearning_Pytorch3DUnet")


['.ipynb_checkpoints',
 '1st_trial',
 '2nd_trial',
 '3rd_trial',
 '4th_trial',
 '5th_trial_elong',
 '5th_trial_elongated_wrongone',
 '6th_trial',
 'notes.txt',
 'Outputs_Excels',
 'segmentation_quality_check.ipynb']

In [5]:
import os
import h5py
import tifffile
import numpy as np

def convert_h5_to_tif_3d(h5_file_path, tif_file_path):
    with h5py.File(h5_file_path, 'r') as h5_file:
        tifffile.imwrite(tif_file_path, h5_file['predictions'], imagej=True) # If gt, put "label"

def process_folders(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for directory in dirs:
            if directory == 'outputPredicted_G':
                dir_path = os.path.join(root, directory)
                for filename in os.listdir(dir_path):
                    if filename.endswith('.h5'):
                        h5_file_path = os.path.join(dir_path, filename)
                        tif_file_path = os.path.join(dir_path, filename.replace('.h5', '.tif'))
                        convert_h5_to_tif_3d(h5_file_path, tif_file_path)

folder_path = "."
process_folders(folder_path)


In [ ]:
#This is done to delete all the cells touching the boundaries of the image, 
#but we are not doing it now, as we believe our model will be good enough.

'''
import tifffile
import numpy as np
from scipy.ndimage import binary_erosion
# Function to remove labels touching the borders
def get_edge_labels(image):
    # Identifying all unique labels in the image
    unique_labels = np.unique(image)
    # Creating a mask for edge-touching labels
    edge_touching = np.zeros_like(image, dtype=bool)
    edge_touching[0, :] = edge_touching[-1, :] = True
    edge_touching[:, 0] = edge_touching[:, -1] = True
    edge_touching_mask = image * edge_touching
    # Identifying labels touching the edges
    edge_labels = np.unique(edge_touching_mask)
    return edge_labels
def remove_labels(image, ids):
    # Removing edge labels
    for label in ids:
        if label != 0:  # Assuming 0 is the background label
            image[image == label] = 0
    return image
# Reading the TIFF file
file_name = "rawWD3.2_21-03_WT_MP_cp_masks.tif"
with tifffile.TiffFile(file_name) as tif:
    images = tif.asarray()

# Processing each image in the stack
#edge_ids = np.concatenate([get_edge_labels(image) for image in images])
#processed_images = np.array([remove_labels(image, edge_ids) for image in images])
# Saving the processed stack to a new TIFF file
output_file_name = "processed_" + file_name
tifffile.imsave(output_file_name, processed_images)
'''

### CELLPOSE to label your ".tiff" files (NOT YET IMPLEMENTED)

In [18]:
import os
from cellpose import models, io
import skimage.io as skio  # Import scikit-image for saving TIFF files

model = models.Cellpose(model_type='cyto2', gpu=True)

def cellpose_labeler(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for directory in dirs:
            if directory == 'outputPredicted':
                dir_path = os.path.join(root, directory)
                for filename in os.listdir(dir_path):
                    if filename.endswith('.tif'):
                        print(filename)
                        x = os.path.join(dir_path, filename)
                        print(x)
                        img = io.imread(x)
                        masks, flows, styles, diams = model.eval(img, diameter=64, channels=None, anisotropy=10, do_3D=True, flow_threshold=0.4, stitch_threshold=0.1, cellprob_threshold=0)
                        
                        # Save the masks and flows as TIFF using skimage.io. Adjust the file names as needed.
                        skio.imsave(os.path.splitext(x)[0] + '_masks.tif', masks)
                        
#model.eval(x, batch_size=8, channels=None, channel_axis=None, z_axis=None, invert=False, normalize=True, diameter=64.0, do_3D=False, anisotropy=None, net_avg=False, augment=False, tile=True, tile_overlap=0.1, resample=True, interp=True, flow_threshold=0.4, cellprob_threshold=0.0, min_size=15, stitch_threshold=0.1, rescale=None, progress=None, model_loaded=False)

# Your folder path
folder_path = r"C:\Users\AdaLovelace\Documents\Data\AlvaroTFM-2.5D-option\AlvaroTFM"
#folder_path = r"C:\Users\titom\Desktop\TFM new stuff\AnalyzingGraph\models"
cellpose_labeler(folder_path)


Disc1_decon_c1_t1.tif
C:\Users\AdaLovelace\Documents\Data\AlvaroTFM-2.5D-option\AlvaroTFM\models\segmentations\outputPredicted\Disc1_decon_c1_t1.tif


100%|████████████████████████████████████████████████████████████████████████████████| 99/99 [00:00<00:00, 3077.85it/s]


Disc1_decon_c1_t5.tif
C:\Users\AdaLovelace\Documents\Data\AlvaroTFM-2.5D-option\AlvaroTFM\models\segmentations\outputPredicted\Disc1_decon_c1_t5.tif


100%|████████████████████████████████████████████████████████████████████████████████| 99/99 [00:00<00:00, 2404.81it/s]


Disc1_decon_c1_t9.tif
C:\Users\AdaLovelace\Documents\Data\AlvaroTFM-2.5D-option\AlvaroTFM\models\segmentations\outputPredicted\Disc1_decon_c1_t9.tif


...

100%|████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 1773.03it/s]
C:\Users\AdaLovelace\AppData\Local\Temp\ipykernel_15908\2726396163.py:21: UserWarning: C:\Users\AdaLovelace\Documents\Data\AlvaroTFM-2.5D-option\AlvaroTFM\models\segmentations\outputPredicted\WD2.2_21-02_WT_confocalonly_masks.tif is a low contrast image
  skio.imsave(os.path.splitext(x)[0] + '_masks.tif', masks)


WD3.2_21-03_WT_MP.tif
C:\Users\AdaLovelace\Documents\Data\AlvaroTFM-2.5D-option\AlvaroTFM\models\segmentations\outputPredicted\WD3.2_21-03_WT_MP.tif


100%|████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 2014.22it/s]
C:\Users\AdaLovelace\AppData\Local\Temp\ipykernel_15908\2726396163.py:21: UserWarning: C:\Users\AdaLovelace\Documents\Data\AlvaroTFM-2.5D-option\AlvaroTFM\models\segmentations\outputPredicted\WD3.2_21-03_WT_MP_masks.tif is a low contrast image
  skio.imsave(os.path.splitext(x)[0] + '_masks.tif', masks)


Disc1_decon_c1_t1_XY_Z12.tif
C:\Users\AdaLovelace\Documents\Data\AlvaroTFM-2.5D-option\AlvaroTFM\models\segmentations\q\outputPredicted\Disc1_decon_c1_t1_XY_Z12.tif


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4155: error: (-215:Assertion failed) inv_scale_x > 0 in function 'cv::resize'


-esta parte del codigo (cellpose automatico) hay que mejorarla pero la dejamos por un momento y ya lo debuggearemos
-una variable llamada "modelo" y aniado esta variable al nombre del fichero (imagen). Poner en la parte que hacemos "# Save the masks and flows as TIFF using skimage.io. Adjust the file names as needed."
-check what and if does anything "anisotropi"



In [56]:
viewer = napari.Viewer()

### Actual Model Comparer (AMC): To extract the object props of all frames (without using the 2D time laps first, as we dont need it if we are not using Napari to obtain the object probs)

In [6]:
import os
import numpy as np
from tifffile import imread

from pathlib import Path

def load_3d_images(folder_path):
    all_images = []
    all_filenames = []
    all_model_names = []
    for root, dirs, files in os.walk(folder_path):
        for directory in dirs:
            if directory == 'outputPredicted_gt':
                dir_path = os.path.join(root, directory)
                for filename in os.listdir(dir_path):
                    if filename.endswith('masks.tif') and not filename.startswith('C1'):
                        all_model_names.append(root)
                        x = os.path.join(dir_path, filename)
                        
                        
                        print(x)
                        
                        # Load the '.tif' image into a NumPy array
                        image = imread(x)
                        all_images.append(image)
                        all_filenames.append(filename)
    return all_images, all_filenames, all_model_names  # Stack all images into a 3D NumPy array
#folder_path = r"/media/maryam/Data/Alvaro/transferLearning_Pytorch3DUnet/pythorchJupyterAutomated"
folder_path = '.'
all_images, all_filenames, all_model_names = load_3d_images(folder_path)


.\5th_trial_elong\ToDo\allimages.983815\outputPredicted_gt\WD1.1_17-03_WT_MP_cp_masks.tif
.\5th_trial_elong\ToDo\allimages.983815\outputPredicted_gt\WD1_15-02_WT_confocalonly_cp_masks.tif
.\5th_trial_elong\ToDo\allimages.983815\outputPredicted_gt\WD2.1_21-02_WT_confocalonly_cp_masks.tif
.\5th_trial_elong\ToDo\allimages.983815\outputPredicted_gt\WD2.2_14-03_WT_MP_cp_masks.tif
.\5th_trial_elong\ToDo\allimages.983815\outputPredicted_gt\WD2.2_21-02_WT_confocalonly_cp_masks.tif
.\5th_trial_elong\ToDo\allimages.983815\outputPredicted_gt\WD3.2_21-03_WT_MP_cp_masks.tif
.\5th_trial_elong\ToDo\v3\outputPredicted_gt\WD1.1_17-03_WT_MP_cp_masks.tif
.\5th_trial_elong\ToDo\v3\outputPredicted_gt\WD1_15-02_WT_confocalonly_cp_masks.tif
.\5th_trial_elong\ToDo\v3\outputPredicted_gt\WD2.1_21-02_WT_confocalonly_cp_masks.tif
.\5th_trial_elong\ToDo\v3\outputPredicted_gt\WD2.2_14-03_WT_MP_cp_masks.tif
.\5th_trial_elong\ToDo\v3\outputPredicted_gt\WD2.2_21-02_WT_confocalonly_cp_masks.tif
.\5th_trial_elong\ToDo\v

In [7]:
import pandas as pd
from skimage import measure
from datetime import datetime


# Assuming 'all_images' contains your 3D image stack loaded from '.tif' files

# Create an empty list to store regionprops for each frame
all_slices = []
summary_info = []

for image_index, image in enumerate(all_images):  # Use enumerate to get both index and image
    print(image.shape)
    region_props_table = pd.DataFrame(columns=["Frame", "Label", "Area", "Perimeter", "Centroid"])
    
    for numFrame in range(image.shape[0]):
        current_regionprops = measure.regionprops(image[numFrame, :, :])
        
        # https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.regionprops
        for object in current_regionprops:
            # Extract object properties
            label = object.label
            area = object.area
            perimeter = object.perimeter
            #centroid = object.centroid  # Returns (row, column) coordinates
            
            # Add object properties to the DataFrame
            region_props_table = pd.concat([region_props_table, pd.DataFrame([{
                "Frame": numFrame,
                "Label": label,
                "Area": area,
                "Perimeter": perimeter,
            }])], ignore_index=True)


    good_cells, bad_cells = select_cells_by_integrity(np.max(region_props_table.Frame), region_props_table.to_numpy(), 95)
    print(good_cells)
    print(bad_cells)
    # Generate a unique filename for each file
    # Format the current date and time and include the image index for even more uniqueness
    current_time = datetime.now().strftime("%m-%d_") + all_filenames[image_index]
    filename = f'output_file_{image_index}_{current_time}.xlsx'

    # Save the DataFrame to an Excel file with the unique filename
    region_props_table.to_excel(filename, sheet_name='Sheet1', index=False)
        
    all_slices.append(region_props_table)
    
    props = measure.regionprops(image)
    avg_vol = np.mean([prop.area for prop in props])
    std_vol = np.std([prop.area for prop in props])

    avg_major_axis = np.mean([prop.axis_major_length for prop in props])
    std_major_axis = np.std([prop.axis_major_length for prop in props])

    avg_diameter = np.mean([prop.equivalent_diameter_area for prop in props])
    std_diameter = np.std([prop.equivalent_diameter_area for prop in props])

    avg_euler = np.mean([prop.euler_number for prop in props])
    std_euler = np.std([prop.euler_number for prop in props])

    avg_extent = np.mean([prop.extent for prop in props])
    std_extent = np.std([prop.extent for prop in props])

    summary_info.append({
        'File Name': filename,
        'Good Cells Count': good_cells,
        'Bad Cells Count': bad_cells,
        'Average Volume' : avg_vol,
        'STD Volume' : std_vol,
        'Average Major Axis' : avg_major_axis,
        'STD Major Axis' : std_major_axis,
        'Average Diameter' : avg_diameter,
        'STD Diameter' : std_diameter,
        'Average Euler' : avg_euler,
        'STD Euler' : std_euler,
        'Average Extent' : avg_extent,
        'STD Extent' : std_extent,
    })


(105, 512, 512)
39
481
(36, 512, 512)
63
269
(38, 512, 512)
33
318
(59, 512, 512)
26
671
(39, 512, 512)
38
352
(60, 512, 512)
38
417
(105, 512, 512)
30
388
(36, 512, 512)
65
179
(38, 512, 512)
42
119
(59, 512, 512)
35
505
(39, 512, 512)
46
207
(60, 512, 512)
24
406


In [8]:
import pandas as pd
import numpy as np

# Assuming 'all_model_names' and 'summary_info' are already defined

# Convert the summary information to a DataFrame
summary_df = pd.DataFrame(summary_info)

# Initially, we try to insert 'Model Name', but if it's already there, we skip this step
if 'Model Name' not in summary_df.columns:
    summary_df.insert(0, 'Model Name', all_model_names[:len(summary_df)])

# Calculate the mean of 'Good Cells Count' and 'Bad Cells Count' for each 'Model Name'
good_cells_avg = summary_df.groupby('Model Name')['Good Cells Count'].transform('mean')
bad_cells_avg = summary_df.groupby('Model Name')['Bad Cells Count'].transform('mean')

# Add the calculated averages as new columns to the right side of the DataFrame
summary_df['Avg Good Cells'] = good_cells_avg
summary_df['Avg Bad Cells'] = bad_cells_avg

# Identifying missing model names
existing_model_names = set(summary_df['Model Name'])
all_model_names_set = set(all_model_names)
missing_model_names = list(all_model_names_set - existing_model_names)

# Creating DataFrame for missing model names
if missing_model_names:
    missing_df = pd.DataFrame(missing_model_names, columns=['Model Name'])
    # Assuming 'Good Cells Count' and 'Bad Cells Count' are important, we fill them with NaN for missing models
    missing_df['Good Cells Count'] = np.nan
    missing_df['Bad Cells Count'] = np.nan
    # Calculate mean values for the NaN columns if applicable, or set as NaN
    missing_df['Avg Good Cells'] = np.nan
    missing_df['Avg Bad Cells'] = np.nan
    # Append the missing models DataFrame to the original summary_df
    summary_df = pd.concat([summary_df, missing_df], ignore_index=True)

# Save the updated summary DataFrame to an Excel file
summary_df.to_excel('summary_file.xlsx', index=False)

# Display missing model names for verification
print("Model names missing from summary_df:", missing_model_names)


Model names missing from summary_df: []


# From here onwards a bit messy and still on progress

In [31]:
region_props_array = region_props_table.to_numpy()


### 1) Load the raw image to Napari
### 2) Load the labels to Napari (if you dont have your predictions as layers, do the "Extra step")
### 3) Convert to the raw image into 2D timelapse using using the plugin "Convert to 2D timelapse"
### 4) Convert to the labels into 2D timelapse using using the plugin "Convert to 2D timelapse"
### 5) Use the tool named "regionprops of all frames" and REMEMBER to use the 2D images (that will create a table with much data, you just have to leave that table as is, and continue running this code)


#### Extra step) Open cellpose y hacer "label segmentations"

In [46]:
print(np.max(region_props_table.Frame))
print(region_props_table.to_numpy())

58
[[0 1 418.0 90.87005768508881 nan]
 [0 2 1102.0 131.88225099390857 nan]
 [0 3 382.0 79.18376618407358 nan]
 ...
 [58 633 841.0 122.94722215136416 nan]
 [58 634 890.0 144.91168824543144 nan]
 [58 635 1131.0 134.5269119345812 nan]]


In [44]:
#ESTE CODIGO ES EL QUE QUEREMOS QUE FUNCIONE, PERO NO ENTIENDO PORQUE NO LO HACE

# extract both "good" (present in at least 85% planes) and "bad" (present in fewer than 15% planes) cell labels  
#labels_layer = viewer.layers[3] #CHANGE EVERY TIME TO THE 2D LABEL
#image_layer = viewer.layers[2] #CHANGE EVERY TIME TO THE 2D RAW IMAGE

good_cells = select_cells_by_integrity(np.max(region_props_table.Frame), region_props_table.to_numpy(), 95, 'good')
#good_ids = [x[0] for x in good_cells]
#good_labels = select_labels_stack(region_props_table.to_numpy(), good_ids)

bad_cells = select_cells_by_integrity(np.max(region_props_table.Frame), region_props_table.to_numpy(), 5, 'bad')
#bad_ids = [x[0] for x in bad_cells]
#bad_labels = select_labels_stack(region_props_table, bad_ids)

print('"Good" labels in dataset =',(len(good_cells)))
print('"Bad" labels in dataset =',(len(bad_cells)))

print('Percentage of "good" labels in dataset =',round((len(good_cells)/len(region_props_table))*100,1),'%')
print('Percentage of "bad" labels in dataset =',round((len(bad_cells)/len(region_props_table))*100,1),'%')

"Good" labels in dataset = 5878
"Bad" labels in dataset = 19
Percentage of "good" labels in dataset = 81.9 %
Percentage of "bad" labels in dataset = 0.3 %


In [41]:
print('"Good" labels in dataset =',(len(good_cells)))
print('"Bad" labels in dataset =',(len(bad_cells)))

"Good" labels in dataset = 300
"Bad" labels in dataset = 0


In [10]:
# add the new labels to the viewer
viewer.add_labels((np.array(good_labels)), name='good cells')
viewer.add_labels((np.array(bad_labels)), name='bad cells')

<Labels layer 'bad cells' at 0x20767215150>

In [15]:
# Note: if labels need to be 3D instead of timelapse they can be reshaped
label = good_labels
label_3d = np.reshape(label, (label.shape[1], label.shape[0], label.shape[2], label.shape[3])) 

AttributeError: 'list' object has no attribute 'shape'